# Efficient Yelp API Calls

Kris Barbier

In [1]:
#Imports
import pandas as pd
import numpy as np
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
#Load yelp_api credentials
with open('/Users/krisbarber/.secret/yelp_api.json') as f:
    login = json.load(f)
    
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
#Instantiate YelpAPI variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [4]:
#Set API call parameters
LOCATION = 'Denver, CO'
TERM = 'Italian'

In [7]:
#Specifying JSON_FILE filename
#Include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Denver_italian.json"
JSON_FILE

'Data/results_in_progress_Denver_italian.json'

In [8]:
#Set function for API call
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [9]:
##Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_Denver_italian.json not found. Saving empty list to new file.
- 0 previous results found.


65

In [10]:
#Use for loop to run API call
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/65 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [11]:
#Load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,jVOAZe9L3Kwx9Ze-UhT3_w,odyssey-italian-restaurant-denver,Odyssey Italian Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/X5Mh_w...,False,https://www.yelp.com/biz/odyssey-italian-resta...,1286,"[{'alias': 'sicilian', 'title': 'Sicilian'}, {...",4.5,"{'latitude': 39.7257806690032, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '603 E 6th Ave', 'address2': '', ...",+13033180102,(303) 318-0102,1352.170004
1,i8VF3q7dTszCcigJSMcJzQ,angelo-s-taverna-denver,Angelo’s Taverna,https://s3-media3.fl.yelpcdn.com/bphoto/xsp2nV...,False,https://www.yelp.com/biz/angelo-s-taverna-denv...,1621,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 39.725523, 'longitude': -104.979224}","[restaurant_reservation, delivery, pickup]",$$,"{'address1': '620 E 6th Ave', 'address2': '', ...",+13037443366,(303) 744-3366,1351.392992
2,Y-c9rFdGYXlT-A1ndoCTrg,tavernetta-denver,Tavernetta,https://s3-media1.fl.yelpcdn.com/bphoto/geY9Y_...,False,https://www.yelp.com/biz/tavernetta-denver?adj...,611,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 39.75302254234153, 'longitude': -...","[delivery, pickup]",$$$,"{'address1': '1889 16th St', 'address2': None,...",+17206051889,(720) 605-1889,3337.975147
3,QAMR-ujpiN4nuggIDMBXaA,difrancos-denver-2,DiFranco's,https://s3-media2.fl.yelpcdn.com/bphoto/6AUea-...,False,https://www.yelp.com/biz/difrancos-denver-2?ad...,576,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 39.731024, 'longitude': -104.9863...","[delivery, pickup]",$$,"{'address1': '955 Lincoln St', 'address2': 'Un...",+17202531244,(720) 253-1244,1464.897305
4,iM8ijTk7rpzcwGAIm7thwg,gattara-denver-2,Gattara,https://s3-media2.fl.yelpcdn.com/bphoto/Piml66...,False,https://www.yelp.com/biz/gattara-denver-2?adju...,103,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 39.744387, 'longitude': -104.983221}","[restaurant_reservation, delivery, pickup]",NaN,"{'address1': '1776 Grant St', 'address2': '', ...",+13033187272,(303) 318-7272,1487.672448


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,FuAU4_bliFhPX_lbVyVedw,big-daddys-pizza-aurora,Big Daddy's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/WNkppG...,False,https://www.yelp.com/biz/big-daddys-pizza-auro...,80,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 39.69575442273203, 'longitude': -...","[pickup, delivery]",NaN,"{'address1': '1155 S Havana St', 'address2': '...",+13036962199,(303) 696-2199,9890.741705
996,pSv-5G1yRA_Nbhy360f-wA,my-neighbor-felix-boulder-boulder,My Neighbor Felix Boulder,https://s3-media1.fl.yelpcdn.com/bphoto/lp6zVx...,False,https://www.yelp.com/biz/my-neighbor-felix-bou...,201,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.5,"{'latitude': 40.01745, 'longitude': -105.28357}",[],$$,"{'address1': '901 Pearl St', 'address2': None,...",+13033769552,(303) 376-9552,41193.017038
997,dY4KTXwRY0cODJUy8g8xWA,glacier-homemade-ice-cream-and-gelato-greenwoo...,Glacier Homemade Ice Cream & Gelato,https://s3-media1.fl.yelpcdn.com/bphoto/5dkhKO...,False,https://www.yelp.com/biz/glacier-homemade-ice-...,107,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",4.5,"{'latitude': 39.625318, 'longitude': -104.883418}","[pickup, delivery]",$,"{'address1': '4940 S Yosemite', 'address2': 'S...",+13032212539,(303) 221-2539,14331.123338
998,JT248b_bTsxaaylP0XN1ow,blackjack-pizza-thornton-2,Blackjack Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/4l5YVf...,False,https://www.yelp.com/biz/blackjack-pizza-thorn...,16,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.9551553, 'longitude': -104.985...","[pickup, delivery]",NaN,"{'address1': '14296 Lincoln St', 'address2': '...",+13034699535,(303) 469-9535,24364.604079
999,8bg6VJHRM2JLSQdGFwYzSA,big-daddys-pizza-denver-2,Big Daddy's Pizza - Bear Valley,https://s3-media1.fl.yelpcdn.com/bphoto/IL5ub9...,False,https://www.yelp.com/biz/big-daddys-pizza-denv...,83,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.0,"{'latitude': 39.6658051013947, 'longitude': -1...","[pickup, delivery]",$,"{'address1': '2772 S Wadsworth Blvd', 'address...",+13039883697,(303) 988-3697,12269.612700


In [12]:
#Check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [13]:
#Save the final results to a compressed csv
final_df.to_csv('Data/final_results_Denver_italian.csv.gz', compression='gzip',index=False)